In [ ]:
import numpy as np
import cv2 
import matplotlib.pyplot as plt
import math

In [ ]:
coin_image_path = "images\coins.png"
sudoku_image_path = "images\sudoku.png"
quote_image_path = "images\quote.png"
coin_image = cv2.imread(coin_image_path)
sudoku_image = cv2.imread(sudoku_image_path)
quote_image = cv2.imread(quote_image_path)

### 1 Histogram Computation

In [ ]:
class Q1:

    def __init__(self, img, K = 256):
        self.img = img
        if (len(self.img.shape) == 3):
            self.img = cv2.cvtColor(self.img, cv2.COLOR_BGR2GRAY)
        self.h, self.w = self.img.shape
        self.freq = np.array(0)
        self.probabilities = np.array(0)
        self.K = K

    def intensity_probabilities(self):
        freq = self.histogram()
        self.probabilities = freq / np.sum(freq)

    def histogram(self):
        freq = np.zeros(256)
        if self.img is not None:
            h, w = self.h, self.w
            for i in range(h):
                for j in range(w):
                    freq[self.img[i, j]] += 1

        self.freq = freq
        return freq
    
    def plot_histogram(self):
        freq = self.histogram()
        plt.plot(freq)
        plt.xlabel("Intensities")
        plt.ylabel("Frequencies")
        plt.show()

    def average_intensity_from_histogram(self):
        avg_intensity = 0
        for i, f in enumerate(self.freq):
            avg_intensity += i * f
        h, w = self.h, self.w
        avg_intensity /= (h * w)
        return avg_intensity
    
    def average_intensity_from_image(self):
        return np.mean(self.img)
    
    def is_intensity_correct_from_histogram(self):
        return self.average_intensity_from_histogram() == self.average_intensity_from_image()       

        

### 2. Otsu’s Binarization

In [ ]:
class Q2(Q1):
    def __init__(self, img, offset = 0):
        super().__init__(self.add_offset(img, offset=offset))
        self.intensity_probabilities()

    def add_offset(self, img, offset = 0):
        offset_image = np.clip(img.astype(np.int16) + offset , 0, 255)            
        return offset_image.astype(np.uint8)


    def class_probabilities(self, t):
        prob = self.probabilities
        class_0_probability = np.sum(prob[:t + 1])
        class_1_probability = np.sum(prob[t + 1:])
        return class_0_probability, class_1_probability
    
    def class_means(self, t, w0, w1):
        prob = self.probabilities
        class_0_mean =  np.sum(np.array([k*prob[k] for k in range(t + 1)])) / w0
        class_1_mean =  np.sum(np.array([k*prob[k] for k in range(t + 1, len(prob))])) / w1
        return class_0_mean, class_1_mean
    
    def within_class_variance(self, t):
        prob = self.probabilities
        w0, w1 = self.class_probabilities(t)
        if (w0 == 0 or w1 == 0): return None
        mu0, mu1 = self.class_means(t, w0, w1)
        var0_sq = np.sum(np.array([(((k - mu0)**2) * prob[k]) for k in range(t + 1)])) / w0
        var1_sq = np.sum(np.array([(((k - mu1)**2) * prob[k]) for k in range(t + 1, len(prob))])) / w1
        var_sq = w0 * var0_sq + w1 * var1_sq
        return var_sq

    def between_class_variance(self, t):
        w0, w1 = self.class_probabilities(t)
        if (w0 == 0 or w1 == 0): return None
        mu0, mu1 = self.class_means(t, w0, w1)
        between_class_variance = w0 * w1 * ((mu0 - mu1) ** 2)
        return between_class_variance



    def minimize_within_class_variance(self):
        t_opt, min_within_class_variance = 0, math.inf
        for t in range(self.K):
            var_sq = self.within_class_variance(t)
            if (var_sq is not None and var_sq < min_within_class_variance):
                min_within_class_variance = var_sq
                t_opt = t
        
        return t_opt        

    def maximize_between_class_variance(self):
        t_opt, max_between_class_variance = 0, 0
        for t in range(self.K):
            between_class_variance = self.between_class_variance(t)
            if (between_class_variance is not None and between_class_variance > max_between_class_variance):
                t_opt = t
                max_between_class_variance = between_class_variance
        return t_opt
    
    def binarize(self, title = None, t_opt = None):
        if t_opt is None:
            t_opt = self.maximize_between_class_variance()
        binarized_image = np.where(self.img > t_opt, 255, 0).astype(np.uint8)
        if title is not None:
            success = cv2.imwrite(f"{title}.png", binarized_image)
            if success:
                print(f"{title} saved successfully")
            cv2.imshow(title, binarized_image)
            cv2.waitKey(0)
        
        return binarized_image



        

### 3. Adaptive Binarization

In [ ]:
class Q3:
    def __init__(self, image, N = None):
        self.img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        self.N = N

    def adaptive_binarization(self):
        binarized_image = np.zeros_like(self.img)
        img = self.img
        N = self.N
        if N is None:
            binarization = Q2(img)
            binarized_image = binarization.binarize()
            success = cv2.imwrite(f"full_binarized_image.png", binarized_image)
            if success:
                print(f"full_binarized_image.png saved successfully")
            cv2.imshow("Full Binarized Image", binarized_image)
            cv2.waitKey(100)
        else:
            H, W = img.shape
            hu = 0
            while hu < H:
                hd = min(H, hu + N )
                wl = 0
                while wl < W:
                    wr = min(W, wl + N)
                    binarization = Q2(img[hu:hd, wl:wr])
                    binarized_image[hu:hd, wl:wr] = np.maximum(binarized_image[hu:hd, wl:wr], binarization.binarize()) # majority vote
                    wl = int(wl + (0.8 * N)) # 20% overlap
                hu = int(hu + (0.8 * N)) # 20% overlap
            cv2.imshow(f"Binarized Image With Blocks of Size {N}", binarized_image)
            cv2.waitKey(0)
            success = cv2.imwrite(f"binarized_image_with_blocks_of_size_{N}.png", binarized_image)
            if success:
                print(f"binarized_image_with_blocks_of_size_{N}.png saved successfully")
        
        return binarized_image




### 4. Connected Components

In [ ]:
class Q4(Q2):

    def __init__(self, img):
        super().__init__(img)
        self.image = img
        self.binarized_image = self.binarize()
        self.visited = np.zeros_like(self.binarized_image)
        self.connected_component = []
        self.largest_connected_component = []

    def dfs(self, i, j):
        if (i < 0 or j < 0 or i >= self.h or j >= self.w or self.visited[i, j]): return
        self.visited[i, j] = 1
        if (self.binarized_image[i, j] == 255): return
        self.connected_component.append((i, j))
        self.dfs(i - 1, j - 1)
        self.dfs(i, j - 1)
        self.dfs(i + 1, j - 1)
        self.dfs(i - 1, j)
        self.dfs(i + 1, j)
        self.dfs(i - 1, j + 1)
        self.dfs(i, j + 1)
        self.dfs(i + 1, j + 1)


    def get_largest_character(self):
        H, W = self.h, self.w
        for i in range(H):
            for j in range(W):
                self.connected_component = []
                self.dfs(i, j)
                if (len(self.connected_component) > len(self.largest_connected_component)):
                    self.largest_connected_component = self.connected_component.copy()


    def color_red(self):
        resultant_img = self.image
        for (i, j) in self.largest_connected_component:
            resultant_img [i, j] = [0, 0, 255]
        cv2.imwrite("Largest_character_marked_red.png", resultant_img)
        cv2.imshow("Largest Character Marked Red", resultant_img)
        cv2.waitKey(0)



In [ ]:
q1 = Q1(coin_image)
q1.plot_histogram()
average_intensity_from_histogram = q1.average_intensity_from_histogram()
is_intensity_correct_from_histogram = q1.is_intensity_correct_from_histogram()
print(f"Average intensity from histogram: {average_intensity_from_histogram: .2f}")
print(f"Is intensity correct from histogram? {is_intensity_correct_from_histogram}")


In [ ]:
q2_a = Q2(coin_image)
gray_coin = cv2.cvtColor(coin_image, cv2.COLOR_BGR2GRAY)
threshold = q2_a.minimize_within_class_variance()
q2_a.binarize("Binarize Original Coins Image")
print(f"Threshold on Minimizing the Within Class Variance is: {threshold}")

In [ ]:

q2_b = Q2(coin_image, offset=20)
threshold = q2_b.maximize_between_class_variance()
q2_b.binarize("Binarize Image With Offset 20")
print(f"Threshold on Maximizing the Between Class Variance after adding the offset is: {threshold}")

In [ ]:
for N in [5, 10, 25, 50, None]:
    q3 = Q3(sudoku_image, N)
    q3.adaptive_binarization()

In [ ]:
q4 = Q4(quote_image)
q4.get_largest_character()
q4.color_red()